In [3]:
import cv2
import dlib
import numpy as np

predictor = dlib.shape_predictor("D:/Computer_Vision_Recap/shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()


def get_facial_landmarks(image):
    # func gets the facial landmark from image
    rects = detector(image, 1)          # gets ROI of the facial landmark
    
    if(len(rects) > 1):
        return "error"
    if(len(rects)) == 0:
        return "error"
    # returns reformatted array for annot landmarks func
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

def annotate_landmarks(image, landmarks):
    # func, numbers the detected landmarks
    image = image.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0,0], point[0,1])
        cv2.putText(image, str(idx), pos,
                   cv2.FONT_HERSHEY_DUPLEX, 0.4, (255, 0, 0), 1)
        cv2.circle(image, pos, 3, (0,0, 255), 1)
    return image


def top_lip(landmarks):
    """
    uses xtracted landmarks arg to get the top lips landmarks
    and ret ,center points
    """
    top_lip_pts = []
    for i in range(50, 53):    # range is landmark of top part of top lip
        top_lip_pts.append(landmarks[i])
    for i in range(61, 64):    # range is landmark points of btm part of top lip
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts, axis = 0)     # gets mid points of all the points for calc
    return int(top_lip_mean[:,1])


def bottom_lip(landmarks):
    """
    uses xtracted landmarks to get the bottom lips landmarks
    and ret center point 
    """
    bottom_lip_pts = []
    for i in range(65, 68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56, 59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis= 0)
    
    return int(bottom_lip_mean[:,1])


def mouth_open(image):
    # calc lip distance btw bottom and top lip points
    landmarks = get_facial_landmarks(image)
    if landmarks == "error":
        return image, 0     # ret plain image and the lip instance is ret as 0 
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center - bottom_lip_center)
    return image_with_landmarks, lip_distance

cap = cv2.VideoCapture(0)
yawns = 0              # holds total num of yawns
yawn_status = False

while True:
    ret, frame = cap.read()
    image_landmarks, lip_distance = mouth_open(frame)
    
    prev_yawn_status = yawn_status
    
    if lip_distance > 30:
        yawn_status = True
        cv2.putText(frame, "Yawn Detected", (50, 450), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 
                   1, (0, 0, 255), 2)
        yawn_count = "Yawn count: " + str(yawns + 1)
        cv2.putText(frame, yawn_count, (50, 50), cv2.FONT_HERSHEY_PLAIN,
                   1, (255, 0, 0), 2)
        
    else:
        yawn_status = False
    if(prev_yawn_status == True and yawn_status == False):
        yawns += 1      # doers updating of yawns count
    cv2.imshow("Yawn Detector", frame)
    
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()


<ipython-input-3-d26c441a3713>:65: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if landmarks == "error":
